In [0]:
import pandas as pd
import numpy  as np

In [2]:
############ Seed Selection ####################################
coke_ols = pd.read_excel('Juices_Ensemble_Data.xlsx',sheet_name= 'AP')   #### load transformed data here

z = coke_ols['VolSales'].dropna().count() - 1
y = z - 3
#############################################################################################
seed_incre = 10
    
pun_coke  = coke_ols   
pun_coke  = pun_coke.reset_index(drop=True)
pun_coke2 = pun_coke.drop(['States','Date'],axis=1)     

x = 0
                                                                       
train = pun_coke2.iloc[x:y, :]
test  = pun_coke2.iloc[y:z, :]
forecast_train = pun_coke2.iloc[x:z, :]
forecast_date  = pun_coke2.iloc[z:, :]
  
train_target  = train[['VolSales']]
train_feature = train.drop(['VolSales'], axis=1)
test_target   = test[['VolSales']]
test_feature  = test.drop(['VolSales'], axis=1) 
################################## Modelling #######################################################
def mean_absolute_percentage_error(y_true, y_pred): 
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)*100)
####################################################################################################
final_result = pd.DataFrame()
df_result    = pd.DataFrame(columns=['Seed_number','RF_mape','ANN_mape'])
seed_overall = []

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
##################################################################################################
def func1(params):
    rf = RandomForestRegressor(random_state=params*seed_incre)
    
    rf.fit(train_feature,train_target)
    Train_pred_rf = rf.predict(train_feature)
    Test_pred_rf  = rf.predict(test_feature)
    mape_train = mean_absolute_percentage_error(train_target['VolSales'], Train_pred_rf)        
    mape_test  = mean_absolute_percentage_error(test_target['VolSales'], Test_pred_rf)        
      
    rf_mape=(((mape_train)*y)+((mape_test)*(z-y)))/(z+1)
    seed_result.append(rf_mape)
    ###################################################################################################
    model_ann = MLPRegressor(hidden_layer_sizes = (256),max_iter=50, batch_size=1,verbose=0,random_state=params*seed_incre)
    model_ann.fit(train_feature, train_target)
    
    Train_pred_ann = model_ann.predict(train_feature)
    Test_pred_ann  = model_ann.predict(test_feature) 
      
    mape_train = mean_absolute_percentage_error(train_target['VolSales'], Train_pred_ann)
    mape_test  = mean_absolute_percentage_error(test_target['VolSales'], Test_pred_ann)

    ann_mape = (((mape_train)*y)+((mape_test)*(z-y)))/(z+1)
    seed_result.append(ann_mape)

    return 
#######################################################################################################
for ee in range(500):
  seed_result = [] 
  seed_result.append(ee*seed_incre)
  func1(ee)
  
  seed_overall.append(seed_result)

final_result=pd.DataFrame(data=seed_overall, columns=df_result.columns)
####################################################################################################
ann_seed = final_result[['ANN_mape']].idxmin() * seed_incre     
rf_seed  = final_result[['RF_mape']].idxmin()  * seed_incre  
####################################################################################################

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=

In [27]:
print(rf_seed)
print(ann_seed)

            0
RF_mape  4680
ANN_mape    2910
dtype: int64


In [0]:
############## All 15 state's sheet names are given################
all_state = ['WB','DE', 'MH', 'AP', 'UP', 'PU', 'KA', 'GU', 'RA', 'MP', 'OR', 'BI', 'CH','HA','TN']
sta_len = len(all_state)   

In [0]:
##### reading the file ##########################################################
for sta in range(sta_len):
  
  ##### RAW data ######################################################################################
  coke_raw = pd.read_excel('CCI_Hydration_Consolidated_Data.xlsx',sheet_name=all_state[sta])    #### load raw data here
  dt_raw   = coke_raw.copy()
  dt_raw   = dt_raw[dt_raw['Date'] >='2015-01-01'].reset_index(drop=True)  ## date change according to model, format:- YYYY-MM-DD
  power_bi = dt_raw.copy()
  power_bi['Category'] = 'Sparkling'                      # enter category name as needed for power BI
  ##### transformed Data ###############################################################################
  coke_ols = pd.read_excel('CCI_Hydration_Ensemble_Data.xlsx',sheet_name=all_state[sta])   #### load transformed data here
  dt_ols   = coke_ols.copy()
  dt_ols   = dt_ols[dt_ols['Date'] >='2015-01-01'].reset_index(drop=True)  ## date change according to model, format:- YYYY-MM-DD
  ##### coefficient data ###############################################################################
  coef_ols = pd.read_excel('Hydration_coeffs.xlsx', sheet_name =all_state[sta])  #### coefficient data here 
  dt_coef  = coef_ols.copy() 
  ######################################################################################################
  seed     = pd.read_excel("Seed_results_Hydration.xlsx",sheet_name=all_state[sta])  
  rf_seed  = seed.at[0,'RF']               # rf  seed
  ann_seed = seed.at[0,'ANN']              # ann seed          
  ############################### Dynamic Coefficients ################################################

  ##########################################################################################################
  date1 = '2019-01-01'       # date from which dynamic callculations be calculated, format = 'YYYY-MM-DD'
  date2 = '2018-01-01'       # year back as that of 'date1' 

  due_to = 4                 # mention number of variables(features no. for elasticity calculation) and used in column selection in Due-to
  trans  = [0,0,1,0]         # this array is for log transformation, enter for main features for which we calculate elasticity
  
  
  ##############################################################################################################################################
  ##############################################################################################################################################
  ####### train and test data for rf+ann model ###########################################
  z = coke_ols['VolSales'].dropna().count() - 1
  y = z - 3
  ######## predicted ols column selection ###########################################
  list_ols  = list(coke_ols.columns.values) 

  col_names = list_ols[2:-1]             # Enter column number here and it should be features
  col_count = len(col_names)

  ################################################################# OLS:- y = mx + c --- code part
  Y = coke_ols['VolSales']

  xy = 0
  for i in range(col_count):
    xy += coef_ols['Estimate'][i+1] * coke_ols[coke_ols.columns[i+2]]

  xy = coef_ols['Estimate'][0] + xy


  ols_mape = np.mean(np.abs ( (Y - xy)/Y ) * 100)          
  print("MAPE OLS:",abs(ols_mape))

  coke_ols['OLS_Predicted'] = xy   
  #######################################################################################################
  ############################# RF + ANN model ##########################################################
  #######################################################################################################
  states_name = coke_ols['States'].unique()
  r2_mape = []

  for i in range(len(states_name)):
      metric_train=[]
      metric_test=[]
      state = states_name[i]
      print('------------------------------------------------------------------------------------------------------')
      print('Processing for state: ',state)
      pun_coke  = coke_ols[coke_ols['States'] == state]   # Give your state name here
      pun_coke  = pun_coke.reset_index( )
      #'Market',add below for 7 new states
      pun_coke2 = pun_coke.drop(['States','index'],axis=1)  
      x=0
      # y=56                                                                        
      # z=59                                                                        
      train_1 = pun_coke2.iloc[x:y, :]
      test_1  = pun_coke2.iloc[y:z, :]
      forecast_train = pun_coke2.iloc[x:z, :]
      forecast_date  = pun_coke2.iloc[z:, :]
      train = train_1.drop(['Date'],axis=1)
      test  = test_1.drop(['Date'],axis=1)
      metric_train.append(str((train.columns).values.tolist()))
      metric_test.append(str((train.columns).values.tolist()))
      metric_train.append('TRAIN')
      metric_test.append('TEST')
      
      train_target  = train[['VolSales']]
      train_feature = train.drop(['VolSales'], axis=1)
      test_target   = test[['VolSales']]
      test_feature  = test.drop(['VolSales'], axis=1)

      
      print(train_feature.shape)
      print(train_target.shape)
      print(test_feature.shape)
      print(test_target.shape)
      
      ################################## Modelling #######################################################
      
      def mean_absolute_percentage_error(y_true, y_pred): 
          y_true, y_pred = np.array(y_true), np.array(y_pred)
          return np.mean(np.abs((y_true - y_pred) / y_true)*100)
      
  ##################################
  ###  
  ################################## Random Forest ###########################################

      from sklearn.ensemble import RandomForestRegressor
      rf = RandomForestRegressor(random_state=rf_seed)    
      rf.fit(train_feature,train_target)
      Train_pred_rf = rf.predict(train_feature) 
      Test_pred_rf  = rf.predict(test_feature)
      ##### Forecasting the results using RF ################
      rf.fit(pun_coke2.iloc[x:z, :].drop(['VolSales','Date'], axis=1),pun_coke2.iloc[x:z, :]['VolSales'])
      pred_rf = rf.predict(pun_coke2.iloc[z:, :].drop(['VolSales','Date'], axis=1))
      
      actual_test=list(test_target['VolSales'])
      actual_train=list(train_target['VolSales'])

      Train_Results=pd.DataFrame(data=actual_train, columns=['Train_Actual'])
      Train_Results.insert(loc=1, column='Train_Pred_RF', value=Train_pred_rf)
      Test_Results=pd.DataFrame(data=actual_test, columns=['Test_Actual'])
      Test_Results.insert(loc=1, column='Test_Pred_RF', value=Test_pred_rf)
      

      print("                           R_square for training data:",rf.score(train_feature,train_target))
      print("                           R_square for testing data:",rf.score(test_feature,test_target))
      print("                           MAPE for training data:",mean_absolute_percentage_error(train_target['VolSales'], Train_pred_rf))
      print("                           MAPE for testing data:",mean_absolute_percentage_error(test_target['VolSales'], Test_pred_rf))
      print(rf.feature_importances_)
      fi=pd.DataFrame(rf.feature_importances_,index=train_feature.columns)
      # print(fi)
      r2_train=rf.score(train_feature,train_target)
      r2_test=rf.score(test_feature,test_target)
      mape_train=mean_absolute_percentage_error(train_target['VolSales'], Train_pred_rf)
      mape_test=mean_absolute_percentage_error(test_target['VolSales'], Test_pred_rf)
      

    
      metric_train.append(r2_train)
      metric_train.append(mape_train)
      metric_test.append(r2_test)
      metric_test.append(mape_test)

      rf_mape = (((mape_train)*y)+((mape_test)*(z-y)))/(z+1)
      print("RF mape:-",rf_mape)
      
      ################################## ANN ##############################################
      from sklearn.metrics import r2_score
      from sklearn.neural_network import MLPRegressor

      model_ann = MLPRegressor(hidden_layer_sizes = (256),max_iter=50, batch_size=1,verbose=0,random_state=ann_seed)
      print(model_ann.get_params(deep=True))
      
      model_ann.fit(train_feature, train_target)

      ##################### Forecasting the results using ANN #############################
      model_ann.fit(pun_coke2.iloc[x:z, :].drop(['VolSales','Date'], axis=1),pun_coke2.iloc[x:z, :]['VolSales'])
      pred_ann = model_ann.predict(pun_coke2.iloc[z:, :].drop(['VolSales','Date'], axis=1))
      
      Train_pred_ann=model_ann.predict(train_feature)
      Test_pred_ann=model_ann.predict(test_feature)
      
      Train_Results.insert(loc=2, column='Train_Pred_ANN', value=Train_pred_ann)
      Test_Results.insert(loc=2, column='Test_Pred_ANN', value=Test_pred_ann)
      
      
      print("                R_square for training data:",r2_score(train_target['VolSales'], Train_pred_ann))
      print("                R_square for testing data:",r2_score(test_target['VolSales'], Test_pred_ann))
      print("                MAPE for training data:",mean_absolute_percentage_error(train_target['VolSales'], Train_pred_ann))
      print("                MAPE for testing data:",mean_absolute_percentage_error(test_target['VolSales'], Test_pred_ann))
      
      
      mape_train=mean_absolute_percentage_error(train_target['VolSales'], Train_pred_ann)
      mape_test=mean_absolute_percentage_error(test_target['VolSales'], Test_pred_ann)

      ann_mape = (((mape_train)*y)+((mape_test)*(z-y)))/(z+1)
      print("ANN mape:-",ann_mape)
  #############################################################################################################################
  #########################################################
  estimatorts_train    = [Train_pred_rf, Train_pred_ann]
  estimatorts_test     = [Test_pred_rf, Test_pred_ann]
  estimatorts_forecast = [pred_rf,pred_ann]

  estimatorts_train    = pd.DataFrame(estimatorts_train).T
  estimatorts_test     = pd.DataFrame(estimatorts_test).T
  estimatorts_forecast = pd.DataFrame(estimatorts_forecast).T

  estimatorts=estimatorts_train.append(estimatorts_test)
  estimatorts=estimatorts.append(estimatorts_forecast)

  estimatorts = estimatorts.rename(columns={0: 'Pred_RF',1:'Pred_ANN'})

  estimatorts=estimatorts.reset_index(drop=True)
  estimatorts['Pred_OLS'] = coke_ols['OLS_Predicted']
  #####################################################################################################
  #################################################################
  ###################################################################################
  w1=(((ols_mape+rf_mape+ann_mape)**2)/(ols_mape)**2)
  w2=(((ols_mape+rf_mape+ann_mape)**2)/(rf_mape)**2)
  w3=(((ols_mape+rf_mape+ann_mape)**2)/(ann_mape)**2)

  w11=w1/(w1+w2+w3)
  w22=w2/(w1+w2+w3)
  w33=w3/(w1+w2+w3)


  estimatorts["Ensemble"] = (w11*estimatorts["Pred_OLS"]) + (w22*estimatorts["Pred_RF"]) + (w33*estimatorts["Pred_ANN"])
  ###########################################################################
  e=estimatorts["Ensemble"]                       
      
  estimatorts = estimatorts.reset_index(drop=True)
  estimatorts["Ensemble"] = estimatorts["Ensemble"].reset_index(drop=True)
  coke_ols['Ensemble']    = estimatorts["Ensemble"]

  e = e.reset_index(drop=True)
  Y = Y.reset_index(drop=True)

  Ensemble_MAPE=np.mean(np.abs((Y-e)/Y)*100)
  print("MAPE Ensemble:",abs(Ensemble_MAPE))
  #######################################################################################################################

  #################### part of DUE_TO ###################################################################################

  #######################################################################################################################

  list_df      = list(dt_raw.columns.values)    # columns number that covers raw data's features
  dt_col_names = list_df[2:due_to+2]   
  dt_col_count = len(dt_col_names)  
  due_to_varb  = len(dt_col_names)              # features being used in Due_tos for loop       
  sale_ind     = pd.DatetimeIndex(dt_raw['Date']).year.nunique()  # no. of times Sales Index calculation should repeat
                                                                                  
  ####################### Due-To calculations #######################################################################
  ###################################################################################################################

  ####################### Due-To calculations #######################################################################

  Due_to = pd.DataFrame()
  ##### Difference YOY  ###########################################################################################################
  for i in range(due_to_varb):
    Due_to[dt_col_names[i]+'_Diff_YOY'] = dt_raw[dt_col_names[i]].diff(periods = 12)
  ##### Difference MOM  ########################################################################################################### 
  for i in range(due_to_varb):
    Due_to[dt_col_names[i]+'_Diff_MOM'] = dt_raw[dt_col_names[i]].diff(periods = 1)
  ##### DT of variables  ##########################################################################################################
  for i in range(due_to_varb):
    Due_to['DT_'+dt_col_names[i]] = (dt_coef['Estimate'][i+1] * dt_ols[dt_ols.columns[i+2]]) - (dt_coef['Estimate'][i+1] * dt_ols[dt_ols.columns[i+2]].shift(12))
    Due_to['DT_'+dt_col_names[i]].fillna(0, inplace=True)
  ##### % DT variables  ############################################################################################################
  for i in range(due_to_varb):
    Due_to['% DT_'+dt_col_names[i]] = ((Due_to['DT_'+dt_col_names[i]]).to_numpy() / (dt_ols['VolSales']).shift(12)) * 100
  #### DT sales index  #############################################################################################################
  import datetime
  dt_raw['month']       = pd.DatetimeIndex(dt_raw['Date']).month
  sales_index           = dt_raw[['Date','month','VolSales']]
  x = sales_index.groupby(['month']).mean()
  dt_raw['Sales']       = pd.DataFrame(np.tile((x['VolSales']/(sales_index['VolSales']).mean()),sale_ind))  
  dt_raw['Sales_Index'] = dt_raw['Sales'].shift(12)
  dt_raw.drop(['Sales'], axis=1, inplace=True)

  for i in range(due_to_varb):
    Due_to['DT_'+dt_col_names[i]+'_using_sales_index'] = dt_raw['Sales_Index'] * Due_to['DT_'+dt_col_names[i]]
    Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].fillna(0, inplace=True)

  for i in range(1):
    Due_to['DT Weather Sales Index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'] + Due_to['DT_'+dt_col_names[i+1]+'_using_sales_index'] 
  ##### DT % sales index  #############################################################################################################
  for i in range(due_to_varb):
    Due_to['DT_%'+dt_col_names[i]+'_sales_index'] = ((Due_to['DT_'+dt_col_names[i]+'_using_sales_index']).to_numpy() / (dt_ols['VolSales']).shift(12)) * 100

  for i in range(1):
    Due_to['DT_%Weather_Sales_Index'] = Due_to['DT_%'+dt_col_names[i]+'_sales_index'] + Due_to['DT_%'+dt_col_names[i+1]+'_sales_index']
  ##### DT temp MOM  ##################################################################################################################
  for i in range(due_to_varb):
    Due_to['DT_'+dt_col_names[i]+'_MOM'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].diff(periods=1)

  Due_to['DT_Weather_MOM'] = Due_to['DT Weather Sales Index'].diff(periods=1)
  ##### DT actualised rolling sums ##################################################################################################################################
  Due_to['actualized']=coke_ols.apply(
	lambda row: row['VolSales'] if ~np.isnan(row['VolSales']) else row['Ensemble'],axis=1)

  Due_to['actualized_3mm'] = Due_to['actualized'].rolling(3).sum().fillna(0)
  Due_to['actualized_6mm'] = Due_to['actualized'].rolling(6).sum().fillna(0)
  ################3MMT's and 6MMT's calc's############################################################################################################################
  Due_to['3MMT_VolSales'] = coke_ols['VolSales'].rolling(3).sum()
  Due_to['3MMT_Ensemble'] = coke_ols['Ensemble'].rolling(3).sum()
  Due_to['3MMT_VolSales'] = coke_ols['VolSales'].rolling(3).sum()
  Due_to['6MMT_Ensemble'] = coke_ols['Ensemble'].rolling(6).sum()
  Due_to['6MMT_VolSales'] = coke_ols['VolSales'].rolling(6).sum()
  Due_to['3MMT_Category'] = dt_raw.iloc[:, [6]].rolling(3).sum()
  Due_to['6MMT_Category'] = dt_raw.iloc[:, [6]].rolling(6).sum()
  intial_rolling = coke_ols.VolSales.rolling(3-1).sum()
  intial_rolling = intial_rolling.shift(1)
  list1 = coke_ols.Ensemble[2:]
  Due_to['3MMT_forecast'] = intial_rolling+list1
  Due_to['Error']=(((Due_to['3MMT_VolSales'].to_numpy())/Due_to['3MMT_forecast'])-1)*100



  ########################### power Bi DT input--part ################################################################

  ########### Diff YOY #####################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'_Diff_YOY'] = Due_to[dt_col_names[i]+'_Diff_YOY']
  ########### Diff MOM #####################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'_Diff_MOM'] = Due_to[dt_col_names[i]+'_Diff_MOM']
  ########### DT_using sales index #########################################################################
  for i in range(due_to_varb):
    power_bi['DT_'+dt_col_names[i]+'_using_sales_index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index']
  # power_bi['DT Weather Sales Index'] = Due_to['DT Weather Sales Index']
  ########### DT_MOM #######################################################################################
  for i in range(due_to_varb):
    power_bi['DT_'+dt_col_names[i]+'_MOM'] = Due_to['DT_'+dt_col_names[i]+'_MOM']
  # power_bi['DT_Weather_MOM'] = Due_to['DT_Weather_MOM']
  ########### DT_% sales index #############################################################################
  for i in range(due_to_varb):
    power_bi['DT_%'+dt_col_names[i]+'_sales_index'] = Due_to['DT_%'+dt_col_names[i]+'_sales_index']
  # power_bi['DT_%Weather_Sales_Index'] = Due_to['DT_%Weather_Sales_Index']
  ##########################################################################################################
  power_bi['Ensemble'] = coke_ols['Ensemble']
  power_bi['Actualised']     = Due_to['actualized']
  power_bi['Actualised_3MMT'] = Due_to['actualized_3mm']
  power_bi['Actualised_6MMT'] = Due_to['actualized_6mm']
  power_bi['3MMT_Ensemble'] = coke_ols['Ensemble'].rolling(3).sum()
  power_bi['3MMT_VolSales'] = coke_ols['VolSales'].rolling(3).sum()
  power_bi['6MMT_Ensemble'] = coke_ols['Ensemble'].rolling(6).sum()
  power_bi['6MMT_VolSales'] = coke_ols['VolSales'].rolling(6).sum()
  power_bi['3MMT_Category'] = dt_raw.iloc[:, [6]].rolling(3).sum()
  power_bi['6MMT_Category'] = dt_raw.iloc[:, [6]].rolling(6).sum()
  power_bi['3MMT_forecast'] = Due_to['3MMT_forecast']
  power_bi['Error'] = Due_to['Error']
  
  ##########################################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'3MMT'] = dt_raw[dt_col_names[i]].rolling(3).sum()

  for i in range(due_to_varb):
    power_bi['3MMT'+dt_col_names[i]+'_using_sales_index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].rolling(3).sum()
  ##############################################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'6MMT'] = dt_raw[dt_col_names[i]].rolling(6).sum()

  for i in range(due_to_varb):
    power_bi['6MMT'+dt_col_names[i]+'_using_sales_index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].rolling(6).sum()
  ##########################################################################################################
  ############ power bi melt--part ###############################################################################
  df_cat = pd.melt(power_bi, id_vars = ['States','Category', 'Date'], var_name='Metrics')
  # df_cat.head(4)

  #################################################################################################################
  ###################################################################################################################################
  ###########################################################################################################################################
  ###########################################################################################################################################
  estimatorts1 = estimatorts

  with pd.ExcelWriter('Hydration_Dec_Results_{}.xlsx'.format(all_state[sta]),datetime_format  = 'YYYY-MM-DD') as writer:  
      coke_ols.to_excel(writer,    index=False, sheet_name = 'Ensemble')        
      estimatorts1.to_excel(writer,index=False, sheet_name = 'RF+ANN+OLS')          
      Due_to.to_excel(writer,      index=False, sheet_name = 'Due_to')          
      power_bi.to_excel(writer,    index=False, sheet_name = 'bi')              
      df_cat.to_excel(writer,      index=False, sheet_name = 'Power_BI')
      

MAPE OLS: 7.460908036789145
------------------------------------------------------------------------------------------------------
Processing for state:  West Bengal
(56, 10)
(56, 1)
(3, 10)
(3, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


                           R_square for training data: 0.9797093117620471
                           R_square for testing data: 0.740404851837959
                           MAPE for training data: 2.7485926117227195
                           MAPE for testing data: 8.262968103933206
[3.32256114e-02 1.16729136e-02 4.00108602e-03 1.97394124e-02
 1.57190171e-02 2.34783454e-01 3.05026308e-04 2.00941864e-04
 4.36278207e-04 6.79916259e-01]
RF mape:- 2.9785015094711986
{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 1, 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': 256, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 50, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 3200, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


                R_square for training data: 0.5714068045834926
                R_square for testing data: 0.6547466463797764
                MAPE for training data: 12.835272624979508
                MAPE for testing data: 3.3964618626377288
ANN mape:- 12.149410876446094
MAPE Ensemble: 3.8959936629016854
MAPE OLS: 6.687492239260062
------------------------------------------------------------------------------------------------------
Processing for state:  Maharashtra
(56, 9)
(56, 1)
(3, 9)
(3, 1)


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


                           R_square for training data: 0.9524293849935144
                           R_square for testing data: -3.2688098660945286
                           MAPE for training data: 4.586532598896072
                           MAPE for testing data: 42.571966952449166
[0.21595304 0.02609349 0.02401029 0.07070591 0.01890625 0.02888652
 0.01636435 0.59908013]
RF mape:- 6.409362106592126
{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 1, 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': 256, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 50, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 4320, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                R_square for training data: 0.6037448644429986
                R_square for testing data: -83.67028880048663
                MAPE for training data: 14.421243271499732
                MAPE for testing data: 30.46443366888754
ANN mape:- 14.983048736844125
MAPE Ensemble: 8.352746241371074


KeyError: ignored

In [0]:
wb = pd.read_excel('Juices_Dec_Results_WB.xlsx',sheet_name='Power_BI') 
de = pd.read_excel('Juices_Dec_Results_DE.xlsx',sheet_name='Power_BI')  
mh = pd.read_excel('Juices_Dec_Results_MH.xlsx',sheet_name='Power_BI') 
ap = pd.read_excel('Juices_Dec_Results_AP.xlsx',sheet_name='Power_BI') 
up = pd.read_excel('Juices_Dec_Results_UP.xlsx',sheet_name='Power_BI') 
pu = pd.read_excel('Juices_Dec_Results_PU.xlsx',sheet_name='Power_BI') 
ka = pd.read_excel('Juices_Dec_Results_KA.xlsx',sheet_name='Power_BI') 
gu = pd.read_excel('Juices_Dec_Results_GU.xlsx',sheet_name='Power_BI') 
ra = pd.read_excel('Juices_Dec_Results_RA.xlsx',sheet_name='Power_BI') 
mp = pd.read_excel('Juices_Dec_Results_MP.xlsx',sheet_name='Power_BI') 
orr = pd.read_excel('Juices_Dec_Results_OR.xlsx',sheet_name='Power_BI') 
bi = pd.read_excel('Juices_Dec_Results_BI.xlsx',sheet_name='Power_BI') 
ch = pd.read_excel('Juices_Dec_Results_CH.xlsx',sheet_name='Power_BI') 
ha = pd.read_excel('Juices_Dec_Results_HA.xlsx',sheet_name='Power_BI') 
tn = pd.read_excel('Juices_Dec_Results_TN.xlsx',sheet_name='Power_BI')

merge1=wb.append(de)
merge2=merge1.append(mh)
merge3=merge2.append(ap)
merge4=merge3.append(up)
merge5=merge4.append(pu)
merge6=merge5.append(ka)
merge7=merge6.append(gu)
merge8=merge7.append(ra)
merge9=merge8.append(mp)
merge10=merge9.append(orr)
merge11=merge10.append(bi)
merge12=merge11.append(ch)
merge13=merge12.append(ha)
merge14=merge13.append(tn)

merge14.to_excel('Juices_Data_Update_for_Dec_Power_BI_Consolidated_new.xlsx',index=False)

In [0]:
##########Hydration Merge######################
wb = pd.read_excel('Hydration_Dec_Results_WB.xlsx',sheet_name='Power_BI') 
# de = pd.read_excel('Hydration_Dec_Results_DE.xlsx',sheet_name='Power_BI')  
mh = pd.read_excel('Hydration_Dec_Results_MH.xlsx',sheet_name='Power_BI') 
ap = pd.read_excel('Hydration_Dec_Results_AP.xlsx',sheet_name='Power_BI') 
up = pd.read_excel('Hydration_Dec_Results_UP.xlsx',sheet_name='Power_BI') 
pu = pd.read_excel('Hydration_Dec_Results_PU.xlsx',sheet_name='Power_BI') 
ka = pd.read_excel('Hydration_Dec_Results_KA.xlsx',sheet_name='Power_BI') 
gu = pd.read_excel('Hydration_Dec_Results_GU.xlsx',sheet_name='Power_BI') 
ra = pd.read_excel('Hydration_Dec_Results_RA.xlsx',sheet_name='Power_BI') 
mp = pd.read_excel('Hydration_Dec_Results_MP.xlsx',sheet_name='Power_BI') 
orr = pd.read_excel('Hydration_Dec_Results_OR.xlsx',sheet_name='Power_BI') 
bi = pd.read_excel('Hydration_Dec_Results_BI.xlsx',sheet_name='Power_BI') 
ch = pd.read_excel('Hydration_Dec_Results_CH.xlsx',sheet_name='Power_BI') 
# ha = pd.read_excel('Hydration_Dec_Results_HA.xlsx',sheet_name='Power_BI') 
tn = pd.read_excel('Hydration_Dec_Results_TN.xlsx',sheet_name='Power_BI')

# merge1=wb.append(de)
merge2=wb.append(mh)
merge3=merge2.append(ap)
merge4=merge3.append(up)
merge5=merge4.append(pu)
merge6=merge5.append(ka)
merge7=merge6.append(gu)
merge8=merge7.append(ra)
merge9=merge8.append(mp)
merge10=merge9.append(orr)
merge11=merge10.append(bi)
merge12=merge11.append(ch)
# merge13=merge12.append(ha)
merge14=merge12.append(tn)

merge14.to_excel('Hydration_Data_Update_for_Dec_Power_BI_Consolidated.xlsx',index=False)

In [0]:
spk = pd.read_excel('Sparkling_Data_Update_for_Dec_Power_BI_Consolidated_new.xlsx')
juices = pd.read_excel('Juices_Data_Update_for_Dec_Power_BI_Consolidated_new.xlsx')
hyd = pd.read_excel('Hydration_Data_Update_for_Dec_Power_BI_Consolidated.xlsx')

consolidated1=spk.append(juices)
consolidated2=consolidated1.append(hyd)
consolidated2.to_excel('Data_Update_for_Dec_Power_BI_Consolidated.xlsx',index=False)

In [0]:
##########Hydration Merge######################
wb = pd.read_excel('Hydration_Dec_Results_WB.xlsx',sheet_name='Power_BI') 
# de = pd.read_excel('Hydration_Dec_Results_DE.xlsx',sheet_name='Power_BI')  
mh = pd.read_excel('Hydration_Dec_Results_MH.xlsx',sheet_name='Power_BI') 
ap = pd.read_excel('Hydration_Dec_Results_AP.xlsx',sheet_name='Power_BI') 
# up = pd.read_excel('Hydration_Dec_Results_UP.xlsx',sheet_name='Power_BI') 
pu = pd.read_excel('Hydration_Dec_Results_PU.xlsx',sheet_name='Power_BI') 
ka = pd.read_excel('Hydration_Dec_Results_KA.xlsx',sheet_name='Power_BI') 
gu = pd.read_excel('Hydration_Dec_Results_GU.xlsx',sheet_name='Power_BI') 
ra = pd.read_excel('Hydration_Dec_Results_RA.xlsx',sheet_name='Power_BI') 
mp = pd.read_excel('Hydration_Dec_Results_MP.xlsx',sheet_name='Power_BI') 
orr = pd.read_excel('Hydration_Dec_Results_OR.xlsx',sheet_name='Power_BI') 
bi = pd.read_excel('Hydration_Dec_Results_BI.xlsx',sheet_name='Power_BI') 
ch = pd.read_excel('Hydration_Dec_Results_CH.xlsx',sheet_name='Power_BI') 
# ha = pd.read_excel('Hydration_Dec_Results_HA.xlsx',sheet_name='Power_BI') 
tn = pd.read_excel('Hydration_Dec_Results_TN.xlsx',sheet_name='Power_BI')

# merge1=wb.append(de)
merge2=wb.append(mh)
merge3=merge2.append(ap)
# merge4=merge3.append(up)
merge5=merge3.append(pu)
merge6=merge5.append(ka)
merge7=merge6.append(gu)
merge8=merge7.append(ra)
merge9=merge8.append(mp)
merge10=merge9.append(orr)
merge11=merge10.append(bi)
merge12=merge11.append(ch)
# merge13=merge12.append(ha)
merge14=merge12.append(tn)

merge14.to_excel('Hydration_Data_Update_for_Dec_Power_BI_Consolidated.xlsx',index=False)


In [0]:
dt_raw.iloc[:, [6]]
# dt_col_names

,Water_Category
0,1.330777e+06
1,1.256797e+06
2,1.701108e+06
3,1.706777e+06
4,1.778233e+06
...,...
65,3.437318e+06
66,3.450371e+06
67,3.463423e+06
68,3.476476e+06


In [0]:
coke_ols.columns

Index(['States', 'Date', 'Tmax', 'Avg_precip', 'log_economy', 'Unemployment',
       'Total_SPK', 'sept', 'oct', 'nov', 'dec', 'VolSales', 'OLS_Predicted',
       'Ensemble'],
      dtype='object')

In [0]:
# Due_to['3MMT_forecast']=(coke_ols['Ensemble'][2:]+coke_ols['VolSales'].rolling(2).sum()).fillna(0)

In [0]:
# window = 3
intial_rolling = coke_ols.VolSales.rolling(3-1).sum()
intial_rolling = intial_rolling.shift(1)
list1 = coke_ols.Ensemble[2:]
Due_to['3MMT_forecast'] = intial_rolling+list1

In [0]:
Due_to['3MMT_forecast'].head(10)

0             NaN
1             NaN
2    1.845805e+07
3    2.046140e+07
4    2.386524e+07
5    2.481868e+07
6    2.470295e+07
7    2.321510e+07
8    2.219870e+07
9    2.204562e+07
Name: 3MMT_forecast, dtype: float64

In [0]:
coke_ols['VolSales']

0     5.859995e+06
1     5.486534e+06
2     7.120563e+06
3     7.876792e+06
4     8.771224e+06
          ...     
65             NaN
66             NaN
67             NaN
68             NaN
69             NaN
Name: VolSales, Length: 70, dtype: float64

In [0]:
Due_to['Error']=(((Due_to['3MMT_VolSales'].to_numpy())/Due_to['3MMT_forecast'])-1)*100

SyntaxError: ignored

In [0]:
Due_to['fore']=coke_ols['VolSales'].rolling(2).sum()

In [0]:
Due_to['fore'].head(10)

0             NaN
1    1.134653e+07
2    1.260710e+07
3    1.499736e+07
4    1.664802e+07
5    1.689115e+07
6    1.591514e+07
7    1.519498e+07
8    1.445087e+07
9    1.462015e+07
Name: fore, dtype: float64

In [0]:
Due_to['aa']=coke_ols['Ensemble'][2:]

In [0]:
Due_to['aa']

0              NaN
1              NaN
2     7.111517e+06
3     7.854306e+06
4     8.867889e+06
          ...     
65    1.249146e+07
66    1.225919e+07
67    1.216322e+07
68    1.209651e+07
69    1.222871e+07
Name: aa, Length: 70, dtype: float64